In [1]:
import os
import glob
import json
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
result_dir = "../train/result/"
target = "with_face_dir_feature/all.json"
with open(os.path.join(result_dir, target), "r") as fr:
    result_all = json.load(fr)

In [2]:
def analyze(result_dir, config_id):
    print("-" * 20 + "{:^50}".format(config_id) + "-" * 20)
    with open(os.path.join(result_dir, config_id, "all.json"), "r") as fr:
        result_all = json.load(fr)
    
    print("fscore of each test set")
    for id, result in sorted(result_all.items()):
        print("%1d: %1.3f" % (int(id), result['test']['fscore']))
        
    print("fscore of all")
    y_all = np.array([], np.int32)
    t_all = np.array([], np.int32)
    for id, result in sorted(result_all.items()):
        y_all = np.hstack((y_all, np.array(result['test']['y'])))
        t_all = np.hstack((t_all, np.array(result['test']['t'])))
    fscore = precision_recall_fscore_support(t_all, y_all)[2]
    print(fscore)
    

In [7]:
analyze(result_dir, "default")
analyze(result_dir, "without40")
analyze(result_dir, "with_face_dir_feature")
analyze(result_dir, "with_face_dir_feature_sgd")
analyze(result_dir, "large_img")
analyze(result_dir, "without_noise")
analyze(result_dir, "simple")
analyze(result_dir, "easy2")

--------------------                     default                      --------------------
fscore of each test set
1: 0.682
2: 0.692
3: 0.712
4: 0.500
5: 0.598
6: 0.716
fscore of all
[0.85598532 0.65602514]
--------------------                    without40                     --------------------
fscore of each test set
1: 0.746
2: 0.677
3: 0.715
4: 0.392
5: 0.660
6: 0.738
fscore of all
[0.83549044 0.66792639]
--------------------              with_face_dir_feature               --------------------
fscore of each test set
1: 0.755
2: 0.608
3: 0.674
4: 0.468
5: 0.603
6: 0.737
fscore of all
[0.85090809 0.6404064 ]
--------------------            with_face_dir_feature_sgd             --------------------
fscore of each test set
1: 0.761
2: 0.672
3: 0.702
4: 0.461
5: 0.559
6: 0.731
fscore of all
[0.84244328 0.64650463]
--------------------                    large_img                     --------------------
fscore of each test set
1: 0.643
2: 0.562
3: 0.540
4: 0.418
5: 0.539
6: 0.697
fsc